# Morpholigical Operators


* specialized kernels that can achieve some effect on the image e.g. blurring, smoothing, noise reduction...
* some operators are very good at reducing black points on a white background (and vice versa)
* some operators are good at erosion and dilation

From [Types of Morphological Operations](https://www.mathworks.com/help/images/morphological-dilation-and-erosion.html#:~:text=Dilation%20adds%20pixels%20to%20the,used%20to%20process%20the%20image._):

**Morphology** is a broad set of image processing operations that process images based on shapes.
Morphological operations apply a *structuring element* to an input image, creating an output image of the same size. 
In a morphological operation, the value of each pixel in the output image is based on a comparison of the corresponding pixel in the input image with its neighbors. **Dilation** adds pixels to the boundaries of objects in an image. **Erosion** removes (erodes) pixels on object boundaries.
The number of pixels added or removed from the objects in an image depends on the size and shape of the structuring element used to process the image. In the morphological dilation and erosion operations, the state of any given pixel in the output image is determined by applying a rule to the corresponding pixel and its neighbors in the input image.

* *structuring element* defines the neighborhood of the pixel of interest (is it only one pixel left and one right or all 8 pixels around); it is like a kernel (a matrix).

### Dilation

The value of the output pixel is the maximum value of all pixels in the neighborhood. 
In a binary image, a pixel is set to 1 if any of the neighboring pixels have the value 1.
Morphological dilation makes objects more visible and fills in small holes in objects.

### Erosion

The value of the output pixel is the minimum value of all pixels in the neighborhood. 
In a binary image, a pixel is set to 0 if any of the neighboring pixels have the value 0.
Morphological erosion removes islands and small objects so that only substantive objects remain.

### Resources:

[Mathematical morphology](https://en.wikipedia.org/wiki/Mathematical_morphology)

[Morphology (©2003 R. Fisher, S. Perkins, A. Walker and E. Wolfart.)](https://homepages.inf.ed.ac.uk/rbf/HIPR2/morops.htm)

[Eroding and Dilating (OpenCV Documentation)](https://docs.opencv.org/3.4/db/df6/tutorial_erosion_dilatation.html)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def load_img():
    blank_img = np.zeros((600, 600))
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(blank_img, text='ABCDE', org=(50, 300), fontFace=font, fontScale=5, color=(255, 255, 255), thickness=25, lineType=cv2.LINE_AA)
    return blank_img

In [ ]:
def display_img(img):
    fig = plt.figure(figsize=(12,10))
    ax = fig.add_subplot(111)
    ax.imshow(img, cmap='gray')

In [ ]:
img = load_img()
display_img(img)

## Erosion

Erosion removes (erodes) pixels on object boundaries. It places a kernel centre over the current pixel, looks for the minimum value in all neighbours and sets that pixel to that minimum value.

[cv2.erode()](https://docs.opencv.org/3.4/d4/d86/group__imgproc__filter.html#gaeb1e0c1033e3f6b891a25d0511362aeb)

dst = cv.erode(src, kernel[, dst[, anchor[, iterations[, borderType[, borderValue]]]]])


In [ ]:
kernel = np.ones((5, 5), dtype=np.uint8)
kernel

In [ ]:
iterations = 5 # try changing it to e.g. 1..3..5
result = cv2.erode(img, kernel, iterations=iterations)

In [ ]:
display_img(result)

## Opening 

Opening is erosion followed by dilation. It can remove a *background* noise.

From [Opening](https://homepages.inf.ed.ac.uk/rbf/HIPR2/open.htm):
* Opening is somewhat like erosion in that it tends to remove some of the foreground (bright) pixels from the edges of regions of foreground pixels. 
* It is less destructive than erosion in general.
* The exact operation is determined by a structuring element. 
* The effect of the operator is to preserve foreground regions that have a similar shape to this structuring element, or that can completely contain the structuring element, while eliminating all other regions of foreground pixels.

From [Opening (morphology)](https://en.wikipedia.org/wiki/Opening_(morphology)):
* In mathematical morphology, opening is the dilation of the erosion of a set A by a structuring element B. (In mathematical morphology, a structuring element is a shape, used to probe or interact with a given image, with the purpose of drawing conclusions on how this shape fits or misses the shapes in the image. It is typically used in morphological operations, such as dilation, erosion, opening, and closing, as well as the hit-or-miss transform.)
* Together with closing, the opening serves in computer vision and image processing as a basic workhorse of morphological noise removal.
* Opening removes small objects from the foreground (usually taken as the bright pixels) of an image, placing them in the background, while closing removes small holes in the foreground, changing small islands of background into foreground. 
* These techniques can also be used to find specific shapes in an image. Opening can be used to find things into which a specific structuring element can fit (edges, corners, ...).
* One can think of B sweeping around the inside of the boundary of A, so that it does not extend beyond the boundary, and shaping the A boundary around the boundary of the element.

[Morphology](http://fourier.eng.hmc.edu/e161/lectures/morphology/node1.html)

In [ ]:
img = load_img()

### Creating a white noise and adding it to the image

In [ ]:
white_noise = np.random.randint(low=0, high=2, size=(600, 600))  # low value is included, high is not so we'll get values of 0 and 1

In [ ]:
display_img(white_noise)

In [ ]:
# Let's add this white noise to our original image.
# Before that we need to make sure that white_noise levels are scaled to the max span of values in the original image. Zero will remain zero but 1 has to become a max value in the original image:
print(f'img.max() = {img.max()}')

In [ ]:
white_noise = white_noise * img.max()
display_img(white_noise)

In [ ]:
noise_img = white_noise + img
display_img(noise_img)

### Removing the noise via opening

So, let's now get rid of this noise

In [ ]:
opening = cv2.morphologyEx(noise_img, cv2.MORPH_OPEN, kernel)
display_img(opening)

In [ ]:
# it is without noise and almost as good as the original image
display_img(img)

## Closing

If we have white shape on black background and there are some patches of black inside white shape, closing will "close" all these "gaps" (black areas) by filling them white.

From [Closing](https://homepages.inf.ed.ac.uk/rbf/HIPR2/close.htm):

* Closing is similar in some ways to dilation in that it tends to enlarge the boundaries of foreground (bright) regions in an image (and shrink background color holes in such regions), but it is less destructive of the original boundary shape. 
* As with other morphological operators, the exact operation is determined by a structuring element. 
* The effect of the operator is to preserve background regions that have a similar shape to this structuring element, or that can completely contain the structuring element, while eliminating all other regions of background pixels.

* Closing is opening performed in reverse. It is defined simply as a dilation followed by an erosion using the same structuring element for both operations. 
* Closing is good at removing a *black noise* (black pixels within area meant to be all white)

### Creating a black noise and adding it to the image

In [ ]:
img = load_img()

In [ ]:
black_noise = np.random.randint(low=0, high=2, size=(600, 600))
black_noise = black_noise * (-img.max())
display_img(black_noise)

In [ ]:
black_noise_img = black_noise + img
display_img(black_noise_img)
black_noise_img

In [ ]:
# Black noise means black pixels within the area that is meant all to be white. 
# In our case we need to have noise (black pixels) only within the (white) letters. 
# Black background should remain the same (all black). 
# set to 0 each element of the matrix which is equal to -255
black_noise_img[black_noise_img == -255] = 0
black_noise_img.min()

In [ ]:
display_img(black_noise_img)

### Removing black noise via closing

In [ ]:
closing = cv2.morphologyEx(black_noise_img, cv2.MORPH_CLOSE, kernel)
display_img(closing)

## Morphological Gradient

It takes a difference between dilation and erosion of an image. Dilation expands white object on black background and erosion shrinks it. Their difference will actually show the edges of the object so this is a very simple way of edge detection.

From [Morphological gradient](https://en.wikipedia.org/wiki/Morphological_gradient):
* A difference between the dilation and the erosion of a given image. 
* It is an image where each pixel value (typically non-negative) indicates the contrast intensity in the close neighborhood of that pixel.
* It is useful for edge detection and segmentation applications.

In [ ]:
img = load_img()

In [ ]:
display_img(img)

In [ ]:
gradient = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel)
display_img(gradient)